In [45]:
import caselawnet
from caselawnet import echrapi as echr
import requests
import pandas as pd

We want to fetch `https://hudoc.echr.coe.int/eng#{%22languageisocode%22:[%22ENG%22,%22FRE%22],%22kpthesaurus%22:[%22210%22],%22documentcollectionid2%22:[%22JUDGMENTS%22,%22DECISIONS%22]}`

In [46]:
kpthesaurus_keyword = '(Art. 1) Jurisdiction of States'
url = echr.buildKeywordUrl(kpthesaurus_keyword)
print(url)

https://hudoc.echr.coe.int/app/query/results?sort=&start=0&length=1000&query=((kpthesaurus="210")) AND ((languageisocode="ENG" OR languageisocode="FRE")) AND ((documentcollectionid="JUDGMENTS" OR documentcollectionid="DECISIONS"))&select=sharepointid,Rank,ECHRRanking,languagenumber,itemid,docname,doctype,application,appno,conclusion,importance,originatingbody,typedescription,kpdate,kpdateAsText,documentcollectionid,documentcollectionid2,languageisocode,extractedappno,isplaceholder,doctypebranch,respondent,ecli,appnoparts,sclappnos,article


In [47]:
kptVal = echr.kpthesaurus[kpthesaurus_keyword]
# languages = ['ENG', 'FRE']
# doc_types = ['JUDGMENTS', 'DECISIONS']

# keyword_filter = '((kpthesaurus="' + kptVal + '"))'
# language_filter = construct_or_clause(languages, 'languageisocode="{}"')
# doc_type_filter = construct_or_clause(doc_types, 'documentcollectionid="{}"')

# filters = [keyword_filter, language_filter, doc_type_filter]
# query = ' AND '.join(filters)
# print(query)

# query_url = hudoc_base + '/app/query/results?sort=&start=0&length=1000&query=' + query + selectColumns
# print(query_url)

In [48]:
nodes = echr.getDataForUrl(url)

In [49]:
print('Initial results: ', len(nodes))

Initial results:  141


In [50]:
nodes[0]

{'abstract': 'Admissible',
 'appno': '18360/91',
 'articles': ['P1-1', 'P1-1-1', '1'],
 'creator': 'ADMISSIBILITY',
 'date': '1999-15-6',
 'ecli': 'ECLI:CE:ECHR:1999:0615DEC001836091',
 'id': 'https://hudoc.echr.coe.int/eng#{"appno":["18360/91"]}',
 'language': 'ENG',
 'refersTo': ['15299/89',
  '6780/74',
  '6950/75',
  '15318/89',
  '8007/77',
  '15300/89',
  '18360/91'],
 'subject': 'P1-1;P1-1-1;1',
 'title': 'ANDREOU v. TURKEY',
 'year': 1999}

In [56]:
# Appno's appear multiple time, mostly because of 2 languages (FR and EN) but sometimes because of different type (decision/judgement)
# Select one node per appno (preferably the english one) and merge the references
id_col = 'appno'
node_dict = {}
for node in nodes:
    id0 = node[id_col]
    if id0 not in node_dict:
        node_dict[id0] = node
    else:
        references = list(set(node_dict[id0]['refersTo'] + node['refersTo']))
        if node['language']=='ENG':
            node_dict[id0] = node
        node_dict[id0]['refersTo'] = references

In [57]:
len(node_dict)

79

In [58]:
nodesAll = node_dict.values()
nodesAll = list(nodesAll) 
print('Total number of cases considered: ', len(nodesAll))

Total number of cases considered:  79


In [59]:
links = []
for node in nodesAll:
    appno = node['appno']
    refers = node['refersTo']
    for refAppNo in refers:
        if refAppNo != appno:
            links.append((appno, refAppNo))
links = list(set(links))
print('In total, out cases contain a total number of citations: ', len(links))

In total, out cases contain a total number of citations:  2009


In [60]:
appnos = [ node['appno'] for node in node_dict.values() ]

filtered_links = []
for source_appno, target_appno in links:
    if (source_appno in appnos) and (target_appno in appnos):
        source_id = node_dict[source_appno]['id']
        target_id = node_dict[target_appno]['id']
        filtered_links.append((source_id, target_id))
print('But we are only interested in the citations inside our set of cases: ', len(filtered_links))

But we are only interested in the citations inside our set of cases:  276


In [61]:
len(filtered_links), len(set(filtered_links))

(276, 276)

In [62]:
def buildEdge(source, target):
    id = source + '_' + target
    return {
      "source": source,
      "target": target,
      "id": id
    }

edges = [ buildEdge(source, target) for source, target in list(set(filtered_links)) ]
nodesAllEnriched = caselawnet.network_analysis.add_network_statistics(nodesAll, edges)

network = {
    'title': kpthesaurus_keyword,
    'nodes': nodesAllEnriched,
    'edges': edges
}

In [63]:
import json
import os
#fnout = 'jurisdiction.json'

fpath = '/media/sf_VBox_Shared/CaseLaw/european/ECtHR/'
fnout = os.path.join(fpath, '2018-10-24-jurisdiction.json')
with open(fnout, 'w') as fout:
    net_str = json.dumps(network)
    fout.write(net_str)

In [64]:
pd.DataFrame(nodesAllEnriched).to_csv(os.path.join(fpath, '2018-10-24-jurisdiction-nodes.csv'), index=False)
pd.DataFrame(edges).to_csv(os.path.join(fpath, '2018-10-24-jurisdiction-edges.csv'), index=False)

## inspect HUDOC output

In [15]:
import requests
import json

resp = requests.get(url)
data = json.loads(resp.text)
nodes = [ hit['columns'] for hit in data['results'] ]

In [16]:
nodes_df = pd.DataFrame(nodes)
nodes_df.head()

,ECHRRanking,Rank,application,appno,appnoparts,article,conclusion,docname,doctype,doctypebranch,...,itemid,kpdate,kpdateAsText,languageisocode,languagenumber,originatingbody,respondent,sclappnos,sharepointid,typedescription
0,1181,6.48195791244507,MS WORD,18360/91,18360;91,P1-1;P1-1-1;1,Admissible,ANDREOU v. TURKEY,HEDEC,ADMISSIBILITY,...,001-4638,6/15/1999 12:00:00 AM,15/06/1999 00:00:00,ENG,1,4,TUR,15318/89,286683,8
1,881,6.48195791244507,TEXT,20652/92,20652;92,1;10;11;11-1;13;35;35-1,Partly inadmissible;Partly admissible,DJAVIT AN v. TURKEY,HEDEC,DECCOMMISSION,...,001-4180,4/14/1998 12:00:00 AM,14/04/1998 00:00:00,ENG,1,3,TUR,21893/93;15318/89,286823,8
2,881,6.48195791244507,TEXT,37650/97,37650;97,1;6;6-1,Inadmissible,MANNERS AND 4 OTHERS v. the United Kingdom,HEDEC,DECCOMMISSION,...,001-4294,5/21/1998 12:00:00 AM,21/05/1998 00:00:00,ENG,1,1,GBR,6878/75;8793/79;9006/80;9310/81;12033/86;17101/90,286940,8
3,881,6.48195791244507,TEXT,13258/87,13258;87,1;6;6-3-c;35;35-3,Inadmissible,M. & Co. v. the FEDERAL REPUBLIC OF GERMANY,HEDEC,DECCOMMISSION,...,001-863,2/9/1990 12:00:00 AM,09/02/1990 00:00:00,ENG,1,21,DEU,14038/88,287348,8
4,901,6.48195791244507,TEXT,15299/89;15300/89;15318/89,15299;89;15300;89;15318;89,1;3;5;5-1;6;7;8;9;13;P1-1;35;35-3;57,Partly admissible;Partly inadmissible,"CHRYSOSTOMOS, PAPACHRYSOSTOMOU AND LOIZIDOU v....",HEDEC,DECCOMMISSION,...,001-2621,3/4/1991 12:00:00 AM,04/03/1991 00:00:00,ENG,1,3,TUR,1474/62;5310/71;6694/74;10328/83,289843,8


In [17]:
nodes_df.doctypebranch.value_counts()

DECCOMMISSION      39
CHAMBER            36
GRANDCHAMBER       30
ADMISSIBILITY      30
DECGRANDCHAMBER     6
Name: doctypebranch, dtype: int64

In [18]:
for c in nodes_df.columns:
    print(c, nodes_df[c].astype(str).nunique())

ECHRRanking 31
Rank 2
application 4
appno 80
appnoparts 80
article 78
conclusion 84
docname 138
doctype 4
doctypebranch 5
documentcollectionid 10
documentcollectionid2 10
ecli 83
extractedappno 94
importance 4
isplaceholder 2
itemid 141
kpdate 74
kpdateAsText 74
languageisocode 2
languagenumber 2
originatingbody 10
respondent 32
sclappnos 76
sharepointid 141
typedescription 5


In [19]:
nodes_df.groupby(['appno', 'docname']).agg(len)['Rank'].sort_values(ascending=False)[:10]

appno     docname                                         
6231/73   HESS v. the UNITED KINGDOM                          2
          HESS c. ROYAUME-UNI                                 2
18360/91  ANDREOU v. TURKEY                                   1
25781/94  CYPRUS v. TURKEY                                    1
          CHYPRE c. TURQUIE                                   1
24833/94  CASE OF MATTHEWS v. THE UNITED KINGDOM              1
          AFFAIRE MATTHEWS c. ROYAUME-UNI                     1
23687/05  CASE OF IVANŢOC AND OTHERS v. MOLDOVA AND RUSSIA    1
          AFFAIRE IVANŢOC ET AUTRES c. MOLDOVA ET RUSSIE      1
9360/81   W. v. IRELAND                                       1
Name: Rank, dtype: int64

In [20]:
nodes_df.groupby(['appno', 'docname']).agg(len)['Rank'].sort_values(ascending=False)[:10]

appno     docname                                         
6231/73   HESS v. the UNITED KINGDOM                          2
          HESS c. ROYAUME-UNI                                 2
18360/91  ANDREOU v. TURKEY                                   1
25781/94  CYPRUS v. TURKEY                                    1
          CHYPRE c. TURQUIE                                   1
24833/94  CASE OF MATTHEWS v. THE UNITED KINGDOM              1
          AFFAIRE MATTHEWS c. ROYAUME-UNI                     1
23687/05  CASE OF IVANŢOC AND OTHERS v. MOLDOVA AND RUSSIA    1
          AFFAIRE IVANŢOC ET AUTRES c. MOLDOVA ET RUSSIE      1
9360/81   W. v. IRELAND                                       1
Name: Rank, dtype: int64

In [55]:
nodes_df.groupby(['appno', 'doctypebranch', 'languageisocode']).agg(len)['Rank'].sort_values(ascending=False)[:10]

appno     doctypebranch  languageisocode
15318/89  GRANDCHAMBER   ENG                2
                         FRE                2
6231/73   DECCOMMISSION  ENG                2
                         FRE                2
22302/10  CHAMBER        FRE                1
22617/07  ADMISSIBILITY  ENG                1
23276/04  ADMISSIBILITY  ENG                1
23687/05  CHAMBER        ENG                1
22302/10  CHAMBER        ENG                1
20652/92  DECCOMMISSION  ENG                1
Name: Rank, dtype: int64

In [41]:
nodes_df.groupby(['ecli', 'languageisocode']).agg(len)['Rank'].sort_values(ascending=False)[:10]

ecli                                languageisocode
ECLI:CE:ECHR:1975:0528DEC000623173  ENG                2
                                    FRE                2
ECLI:CE:ECHR:2018:0717JUD002103405  FRE                1
ECLI:CE:ECHR:1999:0615DEC001836091  ENG                1
ECLI:CE:ECHR:2001:0704DEC004878799  FRE                1
                                    ENG                1
ECLI:CE:ECHR:2000:0208DEC003521497  ENG                1
ECLI:CE:ECHR:2000:0201DEC004135598  ENG                1
ECLI:CE:ECHR:1999:1125DEC005088799  FRE                1
                                    ENG                1
Name: Rank, dtype: int64

In [65]:
nodes_df.groupby('appno').nunique()['doctypebranch'].sort_values(ascending=False)[:10]

appno
20652/92    2
45036/98    2
48787/99    2
9360/81     1
27410/95    1
22617/07    1
23276/04    1
23687/05    1
24833/94    1
25781/94    1
Name: doctypebranch, dtype: int64

In [34]:
nodes_df.groupby(['appno', 'doctypebranch']).nunique()['ecli'].sort_values(ascending=False)[:10]

appno     doctypebranch
15318/89  GRANDCHAMBER     2
9360/81   DECCOMMISSION    1
27765/09  GRANDCHAMBER     1
22617/07  ADMISSIBILITY    1
23276/04  ADMISSIBILITY    1
23687/05  CHAMBER          1
24833/94  GRANDCHAMBER     1
25781/94  DECCOMMISSION    1
27021/08  GRANDCHAMBER     1
27410/95  DECCOMMISSION    1
Name: ecli, dtype: int64

In [42]:
nodes_df.groupby(['ecli']).nunique()['sclappnos'].sort_values(ascending=False)[:10]

ecli
ECLI:CE:ECHR:2005:0303DEC006086100    2
ECLI:CE:ECHR:1990:0312DEC001613790    2
ECLI:CE:ECHR:2002:1212DEC005902100    2
ECLI:CE:ECHR:2009:0421JUD001195607    2
ECLI:CE:ECHR:1999:1125DEC005088799    2
ECLI:CE:ECHR:2012:1019JUD004337004    2
ECLI:CE:ECHR:2013:1126JUD000580908    2
ECLI:CE:ECHR:2003:0220JUD002065292    2
ECLI:CE:ECHR:1983:0228DEC000934881    1
ECLI:CE:ECHR:2001:0904DEC003859597    1
Name: sclappnos, dtype: int64

In [43]:
nodes_df[nodes_df.ecli=='ECLI:CE:ECHR:2005:0303DEC006086100']['sclappnos']

41    4451/70;7151/75;12747/87;13071/87;13134/87;153...
42                                                     
Name: sclappnos, dtype: object

In [21]:
nodes_df.groupby(['ecli', 'docname']).agg(len)['Rank'].sort_values(ascending=False)[:10]

ecli                                docname                                                       
ECLI:CE:ECHR:1975:0528DEC000623173  HESS c. ROYAUME-UNI                                               2
                                    HESS v. the UNITED KINGDOM                                        2
ECLI:CE:ECHR:2018:0717JUD002103405  CASE OF SANDU AND OTHERS v. THE REPUBLIC OF MOLDOVA AND RUSSIA    1
ECLI:CE:ECHR:1999:0615DEC001836091  ANDREOU v. TURKEY                                                 1
ECLI:CE:ECHR:2001:0704DEC004878799  ILAŞCU et AUTRES contre MOLDOVA et RUSSIE                         1
                                    ILAŞCU and OTHERS v. MOLDOVA and RUSSIA                           1
ECLI:CE:ECHR:2000:0208DEC003521497  HAPESHIS AND HAPESHI-MICHAELIDOU v. TURKEY                        1
ECLI:CE:ECHR:2000:0201DEC004135598  GAVRIEL v. TURKEY                                                 1
ECLI:CE:ECHR:1999:1125DEC005088799  YONGHONG v. PORTUGAL             

In [ ]:
nodes_df.groupby(['ecli', 'docname']).agg(len)['Rank'].sort_values(ascending=False)[:10]

In [22]:
nodes_df.groupby(['ecli', 'docname']).agg(len)['Rank'].sort_values(ascending=False)[:10]

ecli                                docname                                                       
ECLI:CE:ECHR:1975:0528DEC000623173  HESS c. ROYAUME-UNI                                               2
                                    HESS v. the UNITED KINGDOM                                        2
ECLI:CE:ECHR:2018:0717JUD002103405  CASE OF SANDU AND OTHERS v. THE REPUBLIC OF MOLDOVA AND RUSSIA    1
ECLI:CE:ECHR:1999:0615DEC001836091  ANDREOU v. TURKEY                                                 1
ECLI:CE:ECHR:2001:0704DEC004878799  ILAŞCU et AUTRES contre MOLDOVA et RUSSIE                         1
                                    ILAŞCU and OTHERS v. MOLDOVA and RUSSIA                           1
ECLI:CE:ECHR:2000:0208DEC003521497  HAPESHIS AND HAPESHI-MICHAELIDOU v. TURKEY                        1
ECLI:CE:ECHR:2000:0201DEC004135598  GAVRIEL v. TURKEY                                                 1
ECLI:CE:ECHR:1999:1125DEC005088799  YONGHONG v. PORTUGAL             

In [23]:
nodes_df[nodes_df.appno=='6231/73'].transpose()

,32,38,39,40
ECHRRanking,881,880,881,880
Rank,6.48195791244507,6.48195791244507,6.48195791244507,6.48195791244507
application,ACROBAT,ACROBAT,ACROBAT,ACROBAT
appno,6231/73,6231/73,6231/73,6231/73
appnoparts,6231;73,6231;73,6231;73,6231;73
article,1,1,1,1
conclusion,Inadmissible,Irrecevable,Inadmissible,Irrecevable
docname,HESS v. the UNITED KINGDOM,HESS c. ROYAUME-UNI,HESS v. the UNITED KINGDOM,HESS c. ROYAUME-UNI
doctype,HEDEC,HFDEC,HEDEC,HFDEC
doctypebranch,DECCOMMISSION,DECCOMMISSION,DECCOMMISSION,DECCOMMISSION


In [24]:
nodes_df[nodes_df.appno=='31322/07'].transpose()

,62
ECHRRanking,1190
Rank,6.48195791244507
application,MS WORD
appno,31322/07
appnoparts,31322;07
article,1;35;35-3
conclusion,Recevable
docname,HAAS c. SUISSE
doctype,HFDEC
doctypebranch,ADMISSIBILITY


In [27]:
nodes_df.to_csv(os.path.join(fpath, '2018-10-24-jurisdiction_nodes_raw.csv'), index=False)